### instant-ngp setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!nvidia-smi

In [ ]:
!apt update && apt install build-essential git python3-dev python3-pip libopenexr-dev libxi-dev libglfw3-dev libglew-dev libomp-dev libxinerama-dev libxcursor-dev
!pip install --upgrade cmake

In [ ]:
!git clone --recursive https://github.com/nvlabs/instant-ngp
%cd instant-ngp
# %cd /content/drive/MyDrive/nvidia/instant-ngp

In [ ]:
#%env TCNN_CUDA_ARCHITECTURES=75
!cmake . -B build -DNGP_BUILD_WITH_GUI=OFF
# !cmake . -B build

In [ ]:
!cmake --build build --config RelWithDebInfo -j `nproc`

In [ ]:
!pip3 install -r requirements.txt

### colmap install







In [ ]:
!sudo apt-get install \
    git \
    cmake \
    build-essential \
    libboost-program-options-dev \
    libboost-filesystem-dev \
    libboost-graph-dev \
    libboost-regex-dev \
    libboost-system-dev \
    libboost-test-dev \
    libeigen3-dev \
    libsuitesparse-dev \
    libfreeimage-dev \
    libgoogle-glog-dev \
    libgflags-dev \
    libglew-dev \
    qtbase5-dev \
    libqt5opengl5-dev \
    libcgal-dev \
    libcgal-qt5-dev

In [ ]:
# Install Ceres-solver (takes 10~20 minutes...)
!sudo apt-get install libatlas-base-dev libsuitesparse-dev
!git clone https://ceres-solver.googlesource.com/ceres-solver
%cd ceres-solver
!git checkout $(git describe --tags) # Checkout the latest release
%mkdir build
%cd build
!cmake .. -DBUILD_TESTING=OFF -DBUILD_EXAMPLES=OFF
!make
!sudo make install

In [ ]:
# Install colmap (takes another 10~20 minutes...)
# Colab Pro P100에서 15분 걸림

!git clone https://github.com/colmap/colmap
%cd colmap
!git checkout dev
%mkdir build
%cd build
!cmake ..
!make
!sudo make install
!CC=/usr/bin/gcc-6 CXX=/usr/bin/g++-6 cmake ..

### Screenshots/Mesh

In [ ]:
# %cd /content/drive/MyDrive/nvidia/instant-ngp
# !bash ./build/testbed --scene data/nerf/fox

In [ ]:
%cd /content/drive/MyDrive/nvidia/instant-ngp
data = "camera" # image folder name
width = 800 # image width
height = 800 # image height

In [ ]:
# make screenshots/mesh file
!python3 scripts/run.py --save_mesh {data}.obj --scene=/content/drive/MyDrive/nvidia/datasets/{data} --mode=nerf --screenshot_transforms=/content/drive/MyDrive/nvidia/datasets/{data}/transforms.json --screenshot_w={width} --screenshot_h={height} --screenshot_dir=/content/drive/MyDrive/nvidia/datasets/{data}/screenshots --save_snapshot=/content/drive/MyDrive/nvidia/datasets/{data}/snapshot.msgpack --n_steps=1000

### colmap with images/videos

In [ ]:
# For small synthetic scenes such as the original NeRF dataset, the default aabb_scale of 1 is ideal 
# For natural scenes, start with an aabb_scale of 16, and subsequently reduce it
# The value can be directly edited in the transforms.json
%cd /content/drive/MyDrive/nvidia/datasets/camera_rm_bg
!python /content/drive/MyDrive/nvidia/instant-ngp/scripts/colmap2nerf.py --colmap_matcher exhaustive --run_colmap --aabb_scale 1

In [ ]:
%cd /content/drive/MyDrive/video
!python /content/drive/MyDrive/nvidia/instant-ngp/scripts/colmap2nerf.py --video_in /content/drive/MyDrive/video/20220131_223648.mp4 --video_fps 3.5 --run_colmap --aabb_scale 8

### vid2img

In [ ]:
%cd /content/drive/MyDrive/vid2img
!python vid2img.py -o /content/drive/MyDrive/nvidia/datasets/tissue/images -i 13 /content/drive/MyDrive/video/20220201_103238.mp4

In [ ]:
!python vid2img.py -o /content/drive/MyDrive/nvidia/datasets/camera/images -i 13 /content/drive/MyDrive/video/20220201_103415.mp4